<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# HLL Flux

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:
* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).
* **(Recommended)** [Del Zanna, Bucciantini, and Londrillo. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics (2003)](https://arxiv.org/abs/astro-ph/0210618).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1. [**The `HLL_flux` function**](#HLL_flux)
    1. [*Function Arguments*](#args)
    1.[*Coordinate-specific Example*](#example)
    1. [*Code Implementation*](#implementation)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The `HLL_flux` function computes the right-hand side of $A_i$. The `calculate_phitilde_and_A_gauge_rhs` function provides the gauge contributions to the right-hand side of the induction equation, while this function calculates the contribution from $B$. For example,

$$
\partial_t A_y = \psi^{6} (v^z B^x - v^x B^z)
$$

<a id='HLL_flux'></a>

# The `HLL_flux` function \[Back to [top](#toc)\]
$$\label{HLL_flux}$$

<a id='args'></a>

## Function Arguments \[Back to [top](#toc)\]
$$\label{args}$$

First, let's look at the argument of `HLL_flux`:
```
double HLL_flux(const A_no_gauge_vars *restrict vars)
```

The output $A^\mathrm{RHS}_i$ is the return value of the function. The only argument for this function is a struct containing stencils for the needed input variables. This struct is defined as
```
typedef struct A_no_gauge_vars {
  double psi6;
  double B1r, B1l;
  double B2r, B2l;
  double c1_min, c1_max;
  double c2_min, c2_max;
  double v1rr, v1rl, v1lr, v1ll;
  double v2rr, v2rl, v2lr, v2ll;
} A_no_gauge_vars;
```
The 1 and 2 portions of the names come from the curl operation which gives the right-hand side. The $r$ and $l$ represent the right/left faces from reconstruction. This is because, usually, $A_i$ is staggered, while $B$ and $v$ are not. Thus, we require the reconstructed values at the faces. As is standard in this documentation, the staggered variables will be denoted by $\underline{A}$.

As an example of the relationship between variables, the staggered vector potential component $\underline{A}_z$ lives at $(i+\frac{1}{2},j+\frac{1}{2},k)$. The velocities are generally only known at cell centers, so they must be reconstructed in both the x and y direction to get the value at $(i+\frac{1}{2},j+\frac{1}{2},k)$. This leads to the notation for `v1` and `v2`. For a reconstructed $\underline{A}_z$, `v1rr` is $v^x$ reconstructed at $(i+\frac{1}{2},j+\frac{1}{2},k)$, taking the right face of both reconstructions. The returned value is $\underline{A}_i^\mathrm{RHS}$. Since the function is direction-agnostic, the value of $i$ is determined by the incoming data, not by anything internal to the function. In addition, the function is unaware of the actual grid structure, so it could also be used to compute $A_i^\mathrm{RHS}$ if the cell-centered vector potential is being evolved.

This function assumes that a staggered quantity $\underline{B}^i$ is known which is staggered only in the direction of $i$. For example, $\underline{B}^x$ is defined at $(i+\frac{1}{2},j,k)$. The $r$ and $\ell$ elements are reconstructed quantities for the single reconstruction of the staggered quantity.

The $c$ elements (e.g. `c1_min`) are the characteristic speeds for that point. These can be computed using the `calculate_characteristic_speed_dirn` functions (for [x](https://github.com/SamuelCupp/GRHayL_beta/blob/main/GRHayL/Flux_Source/calculate_characteristic_speed_dirn0.c), [y](https://github.com/SamuelCupp/GRHayL_beta/blob/main/GRHayL/Flux_Source/calculate_characteristic_speed_dirn1.c), and [z](https://github.com/SamuelCupp/GRHayL_beta/blob/main/GRHayL/Flux_Source/calculate_characteristic_speed_dirn2.c)) in the `Flux_Source` gem. It is important to calculate these values using the properly reconstructed values. For example, the right and left face values for the `dirn0` characteristic speeds should have been reconstructed in the $x$ direction. This is discussed more in their documentation (TODO: link when created).

Finally, the element `psi6` stores the value of the BSSN quantity $\psi^6 = e^{6\phi}$ at the staggered point. Since $\phi$ is naturally cell-centered, this usually requires interpolating to the point before packing the struct.

<a id='example'></a>

## Coordinate-specific Example \[Back to [top](#toc)\]
$$\label{example}$$

Since filling this struct is less trivial than for most functions, we give a specific example assuming that we are calculating the $A_z$ term and using the indexing in
[GRHayL_IGM](https://github.com/SamuelCupp/GRHayL_beta/blob/main/implementations/GRHayL_IGM/src/A_no_gauge_rhs.c)
. In this case,

$$
v_1 = v_x
$$

$$
v_2 = v_y
$$

$$
\underline{B}^1 = \underline{B}^x
$$

$$
\underline{B}^2 = \underline{B}^y
$$

That is to say, all the "1" variables are the $x$ components, and the "2" variables are the $y$ components. Since IllinoisGRMHD (which GRHayL inherits code from) reconstructs to $-\frac{1}{2}$, GRHayL currently also does this. Therefore, we have to increment the reconstructed directions to get the correct face. For the point $(i,j,k)$, the $v$ variables come from the index $(i+1,j+1,k)$. In general, the directions perpendicular to the $i$ of $A_i^\mathrm{RHS}$ are increased by 1. The $\underline{B}^x$ variables are from the index $(i,j+1,k)$, and the point of $\underline{B}^y$ variables are from the point $(i+1,j,k)$. In general, the $B$ variables add 1 to the index $i$ of $\underline{A}_i^\mathrm{RHS}$ permuted by 1. Finally, $\psi^6$ is interpolated to the staggered point. A table summarizing the grid points for these is given below:

|  $\underline{A}_i^\mathrm{RHS}$ Direction  |  $v$ Index  | $\underline{B}_1$ Index  | $\underline{B}_2$ Index  | $\psi^6$ Index  |
|:-----------------:|:-------------:|:-----------:|:-----------:|:-----------------------------------:|
|         x         | $$(i,j+1,k+1)$$ | $$(i,j,k+1)$$ | $$(i,j+1,k)$$ | Interpolated to $$(i,j+\frac{1}{2},k+\frac{1}{2})$$ |
|         y         | $$(i+1,j,k+1)$$ | $$(i+1,j,k)$$ | $$(i,j,k+1)$$ | Interpolated to $$(i+\frac{1}{2},j,k+\frac{1}{2})$$ |
|         z         | $$(i+1,j+1,k)$$ | $$(i,j+1,k)$$ | $$(i+1,j,k)$$ | Interpolated to $$(i+\frac{1}{2},j+\frac{1}{2},k)$$ |

TODO: explanation of characteristic speed indexing in GRHayL_IGM
TODO: this notation also needs to be cleaned up; I don't like the underline, but there aren't many options that don't signify something else

To see a practical example of how to pack this grid function, see [A_no_gauge_rhs.c](https://github.com/SamuelCupp/GRHayL_beta/blob/main/implementations/GRHayL_IGM/src/A_no_gauge_rhs.c) in the `GRHayL_IGM` implementation. This function also demonstrates how to easily produce the desired indices depending on the $A_i^\mathrm{RHS}$ being computed.

<a id='implementation'></a>

## Code Implementation \[Back to [top](#toc)\]
$$\label{implementation}$$

Within the `HLL_flux` function, we first compute the face values for $A_3^\mathrm{RHS}$

$$
\left(A_3^\mathrm{RHS}\right)_{mn} = \psi^6 \left( \left(v_1\right)_{mn}\left(B_2\right)_{m} - \left(v_2\right)_{mn}\left(B_2\right)_{n} \right)
$$

where $m$, $n$ represent the faces $r$ and $\ell$. So, $\left(B_1\right)_r$ is the struct element `B1r`, and $\left(B_2\right)_r$ is the struct element `B2l`.

We also compute the intermediate quantities

$$
\Delta \tilde{B}_1 = \psi^6\left( \left(B_1\right)_{r} - \left(B_1\right)_{l} \right) \\
\Delta \tilde{B}_2 = \psi^6\left( \left(B_2\right)_{r} - \left(B_2\right)_{l} \right)
$$

and

$$
c_1^\mathrm{sum} = c_1^{\min} + c_1^{\max} \\
c_2^\mathrm{sum} = c_2^{\min} + c_2^{\max}
$$

With all of these, we can compute the RHS value using the HLL flux (see Eq. 44 in [Del Zanna, Bucciantini & Londrillo (2003)](https://arxiv.org/abs/astro-ph/0210618))

$$
\underline{A}_3^\mathrm{RHS} = \frac{c_1^{\max} c_2^{\max}}{c_1^\mathrm{sum} c_2^\mathrm{sum}}\left(A_3^\mathrm{RHS}\right)_{ll}
              + \frac{c_1^{\max} c_2^{\min}}{c_1^\mathrm{sum} c_2^\mathrm{sum}} \left(A_3^\mathrm{RHS}\right)_{lr}
              + \frac{c_1^{\min} c_2^{\max}}{c_1^\mathrm{sum} c_2^\mathrm{sum}} \left(A_3^\mathrm{RHS}\right)_{rl}
              + \frac{c_1^{\min} c_2^{\min}}{c_1^\mathrm{sum} c_2^\mathrm{sum}} \left(A_3^\mathrm{RHS}\right)_{rr}
              - \frac{c_1^{\min}c_1^{\max}\Delta \tilde{B}_2}{c_1^\mathrm{sum}}
              + \frac{c_2^{\min}c_2^{\max}\Delta \tilde{B}_1}{c_2^\mathrm{sum}}
$$                                                                                                                  